### **Introduction**


**A model is, abstractly:**

* A function that computes something on tensors (a forward pass)
* Some variables that can be updated in response to training

### **Setup**


In [24]:
import tensorflow as tf
from datetime import datetime
import random as ra
import numpy as np
import tensorboard as tb

print(tb.__version__)
print(tf.__version__)

2.7.0
2.6.0


### **Defining Models and Layers in TensorFlow**
**Layers are functions with a known `mathematical structure` that can be reused and have `trainable variables`.**

In [4]:
# Creating a simple Model using tf.Module
class Module(tf.Module):
    def __init__(self, name):
        super().__init__(name= name)
        self.tr_var = tf.Variable(5.0, name= 'train_me')
        self.non_tr_var = tf.Variable(5.0, trainable=False , name= 'not_train')

    def __call__(self, x):
        return self.tr_var * x + self.non_tr_var

model = Module('SimpleModel')
model(tf.constant(3.)).numpy()

20.0

In [5]:
# Printing trainable & non_trainable variables
print('Trainable variables: ', model.trainable_variables)
print('Non-Trainable variables', model.non_trainable_variables)
print()
print('All variables: ')
for var in model.variables:
    print(var)

Trainable variables:  (<tf.Variable 'train_me:0' shape=() dtype=float32, numpy=5.0>,)
Non-Trainable variables (<tf.Variable 'not_train:0' shape=() dtype=float32, numpy=5.0>,)

All variables: 
<tf.Variable 'not_train:0' shape=() dtype=float32, numpy=5.0>
<tf.Variable 'train_me:0' shape=() dtype=float32, numpy=5.0>


> **Creating a two-layer linear layer model made out of modules.**

In [6]:
# Creating a dense layer
class Dense(tf.Module):
    def __init__(self, in_fea, out_fea,name=None):
        super().__init__(name)
        self.wei = tf.Variable(
            tf.random.normal([in_fea, out_fea], name= 'weights')
        )
        self.bias = tf.Variable(
            tf.zeros([out_fea], name='bias')
        )

    def __call__(self, x_ten):
        # Computation: Y = Matrix multiplication (x & weights) plus the model bias
        # Computation: tf.nn.relu(Y)
        y = (x_ten @ self.wei) + self.bias
        return tf.nn.relu(y)

In [7]:
class Sequential(tf.Module):
    def __init__(self, name=None):
        super().__init__(name)
        
        self.dense1 = Dense(3, 3, 'InputLayer')
        self.dense2 = Dense(3, 2, 'OutputLayer')
        
    def __call__(self, tx):
        y = self.dense1(tx)
        return self.dense2(y)

my_model = Sequential('AI_Model')
# Model acceptable shape [n x 3]
my_model(tf.random.poisson([2, 3], 5.5)) 
my_model(tf.random.normal([3, 3], mean=17, stddev=3.5)).numpy()


array([[1.0707111 , 0.        ],
       [0.        , 0.38128853],
       [0.        , 1.5856438 ]], dtype=float32)

In [8]:
# Printing Model submodules
for mod in my_model.submodules:
    print(mod)
print()

# Printing model Variables
for var in  my_model.variables: 
    print(f'var: {var.numpy()}\n')




var: [0. 0. 0.]

var: [[-1.774076    0.59840745  0.21302606]
 [-0.3786077  -1.1040999  -0.96917003]
 [ 3.1212523  -0.48174986  1.1038833 ]]

var: [0. 0.]

var: [[ 0.4495117 -0.306142 ]
 [ 1.7382188  1.2382319]
 [-1.3513367  0.6562064]]



* ##### **Waiting to create variables**


In [9]:
# The Input shape of the layers is dynamically inferered at the runtime by the size of the incoming tensor..
class dyn_Layer(tf.Module):
    def __init__(self, outCh, name=None):
        super().__init__(name)
        self.is_built = False 
        self.out_channels = outCh
    
    @tf.function
    def __call__(self, tx):
        if not self.is_built:
            self.is_built = True
            self.wei = tf.Variable(
                tf.random.normal([tx.shape[-1], self.out_channels]), name='Weights'
            )
            self.bias = tf.Variable(
                tf.zeros([self.out_channels], name='bias')
            )
        y = tx @ self.wei + self.bias
        return tf.nn.relu(y)

class dyn_SeqModel(tf.Module):
    def __init__(self, name=None):
        super().__init__(name)
        self.dense1 = dyn_Layer(12, 'Input_Layer')
        self.dense2 = dyn_Layer(3, 'Output_Layer')
        
    @tf.function
    def __call__(self, tx):
        tx = self.dense1(tx)
        return self.dense2(tx)

In [10]:
# Calling the dynamic model on different sets of inputs
for i in range(10):
    in_chs = ra.randint(1,15)
    print('InChanels for this Model', in_chs)
    model = dyn_SeqModel('AdaptiveModel')
    tx = tf.random.uniform([6, 6], minval=16, maxval=32)
    print(model(tx))

# Our model is working...



InChanels for this Model 11
tf.Tensor(
[[  0.        0.      235.57759]
 [  0.        0.      187.48181]
 [  0.        0.      281.76666]
 [  0.        0.      227.95454]
 [  0.        0.      257.15887]
 [  0.        0.      228.90312]], shape=(6, 3), dtype=float32)
InChanels for this Model 15
tf.Tensor(
[[ 81.58479   48.32749  365.38065 ]
 [ 86.1093    20.791538 321.01498 ]
 [ 62.526337  35.49463  291.2561  ]
 [ 75.085625  47.113598 351.09882 ]
 [ 80.2058     0.       260.7804  ]
 [ 82.53196    8.104778 360.18277 ]], shape=(6, 3), dtype=float32)
InChanels for this Model 4
tf.Tensor(
[[  0.       165.62148   44.378143]
 [  0.       158.35931   17.842419]
 [  0.       168.06741   19.492168]
 [  0.       129.34972    0.      ]
 [  0.       171.94539   39.930893]
 [  0.       138.28317    0.      ]], shape=(6, 3), dtype=float32)
InChanels for this Model 1
tf.Tensor(
[[ 38.845837    0.         98.548706 ]
 [143.84264     0.        113.61571  ]
 [ 25.82536    12.375169   83.284775 ]
 [ 37.

### **Saving Weights**

In [11]:
# Create a dir to save weights
chk_path = 'Weights_checkpoint'
checkpoint = tf.train.Checkpoint(model=model)

# Using this Command, the weights are stored into the file.
checkpoint.write(chk_path)
tf.train.list_variables(chk_path)

# Creating a new Model and assigning the `SAME WEIGHTS`
new_mod = dyn_SeqModel('Dynamic_Seq2')
checkpoint2 = tf.train.Checkpoint(model=new_mod)

# Reloading the previous weights
checkpoint2.restore('Weights_checkpoint')

# Testing the new Model on previous input, should be able to produce same output
print(f'Previous Model results:\n {model(tx).numpy()}\n')
print(f'New Model result after restoring weights:\n {new_mod(tx).numpy()}')

print("It worked")


Previous Model results:
 [[0.         0.         0.        ]
 [0.78588104 0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.         0.        ]]

New Model result after restoring weights:
 [[0.         0.         0.        ]
 [0.78588104 0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.         0.        ]]
It worked


### **Saving Functions + Visualization using Tensorboard**

In [12]:
# setting up logging
timest = datetime.now().strftime("%Y%m%d-%H%M%S")
logs_dir = 'logs/func/%s' % timest
writer = tf.summary.create_file_writer(logs_dir)

# Create new model to get fresh trace
SeqModel = dyn_SeqModel('Traced_Model')
tf.summary.trace_on(graph= True)
tf.profiler.experimental.start(logs_dir)

# Only call one tf.function while tracing
print(SeqModel(tf.random.uniform(shape=[4, 9], minval= 50, maxval= 70)))

with writer.as_default():
    tf.summary.trace_export(
        name= 'Function Trace',
        step= 0,
        profiler_outdir= logs_dir
    )

tf.Tensor(
[[ 81.25644    0.         0.      ]
 [ 16.028282   0.         0.      ]
 [ 83.058716   0.         0.      ]
 [130.58557    0.         0.      ]], shape=(4, 3), dtype=float32)


> **Starting Tensorboard to visualize tracing results**

In [13]:
%tensorboard --logs/func

# Masha-Allah it worked

UsageError: Line magic function `%tensorboard` not found.


* ##### **Creating a saved Model**
> The recommended way of sharing completely trained models is to use SavedModel. SavedModel contains both a collection of functions and a collection of weights.

In [14]:
# Saving the Model.. A separate directory is created to store the saved Model 
tf.saved_model.save(SeqModel, 'SavedModel')

# Reloading the saved Model. It is in Graph format with no knowledge of internal TensorFlow code.
SeqMod2 = tf.saved_model.load('SavedModel')

# Checking if the loaded model is an object of Sequential class 
isinstance(SeqMod2, dyn_SeqModel)


INFO:tensorflow:Assets written to: SavedModel\assets


False

> **The new model can only work with `pre-defined signatures` (input shape && data-type). It `cannot` be altered to perform on new signatures like Python Code.**

In [15]:
try:
    print(tx)
    # Sending in a known signature
    print(SeqMod2(tf.random.uniform([4,9])).numpy())
    
    # Sending an unknown argument
    print(SeqMod2(tf.constant(1.0)))
    
except:
    print("Error Raised due to incompatible input signature!!\n")


tf.Tensor(
[[19.372972 24.694708 23.173527 18.843626 26.308035 31.051912]
 [18.846718 28.704638 18.931368 27.646376 24.772425 25.513197]
 [19.655659 24.778566 30.359911 17.207092 21.801762 18.40712 ]
 [26.617722 29.929058 22.761803 27.37734  22.628262 22.63971 ]
 [30.373575 24.196802 26.882948 27.649286 19.97745  30.08856 ]
 [16.944263 23.143307 20.68682  25.31832  29.440924 24.941862]], shape=(6, 6), dtype=float32)
[[4.6377244  0.         0.        ]
 [3.25116    0.         0.        ]
 [1.2656194  0.         0.        ]
 [5.615899   0.         0.13157356]]
Error Raised due to incompatible input signature!!



### **Keras Models and Layers**

* ##### **Keras Layers**
> **Keras Layers as based on TF.Module. One can easily swap out TF.Module with Keras.layers.Layer**

In [16]:
class KerasDense(tf.keras.layers.Layer):
    # In_features defines the number of rows for the Layer_Weights Matrix 
    # Out_features determines the number of columns for the Weights Matrix
    def __init__(self, in_features, out_features, **args):
        super().__init__(**args)
        self.w = tf.Variable(
            tf.random.normal([in_features, out_features], name='Lyr_Weights')
        )
        # Here we are setting the model bias to zero (will try with identity matrix soon)..
        self.b = tf.Variable(
            tf.zeros([out_features], name='Lyr_Bias')
        )
    @tf.function
    def call(self, ten_x):
        return tf.nn.relu((ten_x @ self.w) + self.b)


denselayer = KerasDense(6, 5, name="Simple_Layer")
denselayer(tf.random.uniform([3, 6], minval=10, maxval=20))

<tf.Tensor: shape=(3, 5), dtype=float32, numpy=
array([[ 0.      ,  0.      ,  0.      , 42.338474,  0.      ],
       [ 0.      ,  0.      ,  0.      , 27.794685,  0.      ],
       [ 0.      ,  0.      ,  0.      , 43.56392 ,  0.      ]],
      dtype=float32)>

* ##### **The build step**


In [25]:
# Adding the build function to out Keras based Layer
# Called exactly once.
# Used to adapt Variables to the input shape

class dyn_KerasDense(tf.keras.layers.Layer):
    def __init__(self, out_chs, **kwargs):
        # in the constructor, only call 
        # the parent constructor and 
        # define the output-channels
        super().__init__(**kwargs)
        self.out_features = out_chs
        
    def build(self, in_shape):
        print(f'Input_features: {in_shape[-1]}')
        self.w = tf.Variable(
            tf.random.normal([in_shape[-1], self.out_features]), 
            name='Lyr_Weights'
        )
        self.b = tf.Variable(
            tf.zeros([self.out_features]), 
            name='Lyr_bias'
        )
    
    def call(self, ten_x):
        y = (ten_x @ self.w) + self.b
        return tf.nn.sigmoid(y)
    
# Instantiating a flexible layer
flexible_dense = dyn_KerasDense(out_chs= 4)
# Checking variables to ensure none have been initialized by now 
print("Printing Variables: ",flexible_dense.variables)

# Call the layer and recheck the variables
# Calling the layer implicitly calls build method
# which adapts the Layer to the incoming input_shape 

# For incoming tensor (m x n) && weights Matrix (in_F x out_F)
# Expected Output shape (m x out_features)
tx = tf.constant(
    [
        [2., 3., 4., 7.],
        [8., 6., 3., 5.],
        [5., 2., 7., 1.]
    ]
)

output_ten = flexible_dense(tx)
print(output_ten)
highest_prob = np.argmax(output_ten)
print(f'highest Probability tensor: {highest_prob}')

Printing Variables:  []
Input_features: 4
tf.Tensor(
[[0.99878025 0.99550277 0.96360517 0.00484243]
 [0.96552706 0.99653614 0.9768597  0.00450647]
 [0.93478215 0.8688572  0.99999976 0.23803753]], shape=(3, 4), dtype=float32)
highest Probability tensor: 10


* ##### **Keras Models**


### **Saving Keras Model**

### **`Sonnet`: Alternate deep-learning API for building ML Models**
> **By DeepMind,**
> **Based on tf.Module**